In [ ]:
%matplotlib inline

import pandas as pd
import os
import csv
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn import manifold, datasets
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

In [ ]:
cur_dir = os.path.dirname('__file__')
data = pd.read_csv(os.path.join(cur_dir, "WDI_Data.csv"))

In [ ]:
# This function creates a 21st column, which is the average of a certain stat since the year 2000.
def preprocess(df):
    df = df.copy()
    df['21st'] = df.ix[:, '2000':].mean(axis=1)
    return df

In [ ]:
def one_year(data, year='21st'):
    grouped = data.groupby('Indicator Name')
    indic_dict = {}
    for indicator, group in grouped:
        for index, row in group.iterrows():
            if indicator not in indic_dict:
                indic_dict[indicator] = []
            indic_dict[indicator].append(row[year])
    for indicator, group in grouped:
        names = group['Country Name'].tolist()
        break   
        
    return pd.DataFrame(indic_dict, index=names)


def only_countries(df, n=None):
    if n is None:
        n = len(df)
    words = ['World', 'income', '(developing only)', 'OECD', 'countries', 'Euro', 'Asia', 'America', 'situations', 'states']
    sel = []
    for i in df.index:
        temp = True
        for w in words:
            if w in i:
                temp = False
        sel.append(temp)
        
    return df[sel].iloc[:n]

def few_na_cols(df, thresh=0, required_countries=None):
    if required_countries is None:
        required_countries = df.index
    res = []
    for c in df.columns:
        perc = float(df[c].notnull().sum()) / len(df)
        meets_required = True
        for count in required_countries:
            if df[c].isnull().loc[count]:
                meets_required = False
        if perc >= thresh and meets_required:
            res.append(c)
    return df[res]

In [ ]:
data = preprocess(data)

In [ ]:
df = one_year(data)
# print df.index

In [ ]:
null_pcts = []

for column in df.columns:
    col = df[column]
    null_pct = float(col.notnull().sum()) / len(col)
    null_pcts.append(null_pct)

    
temp = pd.DataFrame(data=null_pcts, index=df.columns,
                    columns=['Data Availability']).sort_values('Data Availability', ascending=False)
print temp.head()

In [ ]:
# call this function after creating a csv for a single country, for a single year, for a group of indicators.
# this function exports the indicator names to a csv.
def create_group_csv(csv_dwnld, save_to):

    data = pd.read_csv(os.path.join(cur_dir, csv_dwnld))
    education_stats = data['Series Name'].tolist()

    with open(save_to, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(education_stats)
        
## And then this gets the corresponding indicator list.
def get_indicator_list_from_csv(csv_file):
    with open(csv_file) as f:
        reader = csv.reader(f)
        your_list = list(reader)
    l = your_list[0]
    l = [s for s in l if s != "nan"]
    return l

## example usage
# create_group_csv('Data_Extract_From_World_Development_Indicators_Data.csv', 'health.csv')
# health_indicators = get_indicator_list_from_csv('health.csv')

In [ ]:
def highest_pop_df(df, n=None, by='Population, total'):
    if n is None:
        n = len(df)
    df = df.sort_values(by,ascending=False)
    return df.iloc[:n]

In [ ]:
def cluster_label_dict(countries, clustering):
    label_dict = {}
    for index, label in enumerate(clustering.labels_):
        if label not in label_dict:
            label_dict[label] = []    
        label_dict[label].append(countries[index])
    return label_dict

In [ ]:
features = {}
features['Simple'] = ['Population, total', 'CO2 emissions (kt)', 'Net migration', 'Trade (% of GDP)']

groups = ['health', 'education', 'economics']

for g in groups:
    features[g] = get_indicator_list_from_csv(g + '.csv')

In [ ]:
highest_pop = highest_pop_df(df)

countries = only_countries(highest_pop, n=30)
economics = few_na_cols(countries[features['economics']])

sample = economics.dropna()
countries = sample.index.tolist()

X = sample.as_matrix()
X = StandardScaler().fit_transform(X)
n_samples, n_features = X.shape

X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X)

linkage = 'ward'
clustering = AgglomerativeClustering(linkage=linkage, n_clusters=6)
clustering.fit(X_red)
clust_dict = cluster_label_dict(countries, clustering)

In [ ]:
def plot_clustering(X, labels, y, title=None):
    X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X)
    x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
    X_red = (X_red - x_min) / (x_max - x_min)
    for i in range(X_red.shape[0]):
        plt.text(X_red[i, 0], X_red[i, 1], str(y[i])[:2],
                 color=plt.cm.spectral(labels[i] / float(len(set(labels)) - 1)),
                 fontdict={'weight': 'bold', 'size': 15})

    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title, size=17)
    plt.axis('off')
    plt.tight_layout()

In [ ]:
plot_clustering(X, clustering.labels_, countries)
plt.gcf().set_size_inches(16, 8)